# Scaling up ML using Cloud ML Engine

In this notebook, we take a previously developed TensorFlow model to predict taxifare rides and package it up so that it can be run in Cloud MLE. For now, we'll run this on a small dataset. The model that was developed is rather simplistic, and therefore, the accuracy of the model is not great either.  However, this notebook illustrates *how* to package up a TensorFlow model to run it within Cloud ML. 

Later in the course, we will look at ways to make a more effective machine learning model.

## Environment variables for project and bucket

Note that:
<ol>
<li> Your project id is the *unique* string that identifies your project (not the project name). You can find this from the GCP Console dashboard's Home page.  My dashboard reads:  <b>Project ID:</b> cloud-training-demos </li>
<li> Cloud training often involves saving and restoring model files. If you don't have a bucket already, I suggest that you create one from the GCP console (because it will dynamically check whether the bucket name you want is available). A common pattern is to prefix the bucket name by the project id, so that it is unique. Also, for cost reasons, you might want to use a single region bucket. </li>
</ol>
<b>Change the cell below</b> to reflect your Project ID and bucket name.


In [1]:
import os
PROJECT = 'qwiklabs-gcp-12b4b0f730830df4' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'qwiklabs-gcp-12b4b0f730830df4' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

In [2]:
# For Python Code
# Model Info
MODEL_NAME = 'stackoverflow'
# Model Version
MODEL_VERSION = 'v1'
# Training Directory name
TRAINING_DIR = 'stackoverflow_trained'

In [21]:
# For Bash Code
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['MODEL_NAME'] = MODEL_NAME
os.environ['MODEL_VERSION'] = MODEL_VERSION
os.environ['TRAINING_DIR'] = TRAINING_DIR 
os.environ['TFVERSION'] = '1.8'  # Tensorflow version

In [4]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


### Create the bucket to store model and training data for deploying to Google Cloud Machine Learning Engine Component

In [5]:
%%bash
# The bucket needs to exist for the gsutil commands in next cell to work
gsutil mb -p ${PROJECT} gs://${BUCKET}

Creating gs://qwiklabs-gcp-12b4b0f730830df4/...
ServiceException: 409 Bucket qwiklabs-gcp-12b4b0f730830df4 already exists.


### Enable the Cloud Machine Learning Engine API

The next command works with Cloud Machine Learning Engine API.  In order for the command to work, you must enable the API using the Cloud Console UI.   Use this [link.](https://console.cloud.google.com/project/_/apis/library)  Then search the API list for Cloud Machine Learning and enable the API before executing the next cell.

Allow the Cloud ML Engine service account to read/write to the bucket containing training data.

In [6]:
%%bash
# This command will fail if the Cloud Machine Learning Engine API is not enabled using the link above.
echo "Getting the service account email associated with the Cloud Machine Learning Engine API"

AUTH_TOKEN=$(gcloud auth print-access-token)
SVC_ACCOUNT=$(curl -X GET -H "Content-Type: application/json" \
    -H "Authorization: Bearer $AUTH_TOKEN" \
    https://ml.googleapis.com/v1/projects/${PROJECT}:getConfig \
    | python -c "import json; import sys; response = json.load(sys.stdin); \
    print (response['serviceAccount'])")  # If this command fails, the Cloud Machine Learning Engine API has not been enabled above.

echo "Authorizing the Cloud ML Service account $SVC_ACCOUNT to access files in $BUCKET"
gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$BUCKET   
gsutil -m acl ch -u $SVC_ACCOUNT:R -r gs://$BUCKET   # error message (if bucket is empty) can be ignored.  
gsutil -m acl ch -u $SVC_ACCOUNT:W gs://$BUCKET      

Getting the service account email associated with the Cloud Machine Learning Engine API
Authorizing the Cloud ML Service account service-40233307502@cloud-ml.google.com.iam.gserviceaccount.com to access files in qwiklabs-gcp-12b4b0f730830df4


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   234    0   234    0     0    950      0 --:--:-- --:--:-- --:--:--   951
No changes to gs://qwiklabs-gcp-12b4b0f730830df4/
No changes to gs://qwiklabs-gcp-12b4b0f730830df4/datalab-backups/us-central1-a/mydatalabvm/content/daily-20190521181602
No changes to gs://qwiklabs-gcp-12b4b0f730830df4/datalab-backups/us-central1-a/mydatalabvm/content/hourly-20190521181602
No changes to gs://qwiklabs-gcp-12b4b0f730830df4/datalab-backups/us-central1-a/mydatalabvm/content/weekly-20190521181602
No changes to gs://qwiklabs-gcp-12b4b0f730830df4/stackoverflow/dataset/stackoverflow-test-10.csv
No changes to gs://qwiklabs-gcp-12b4b0f730830df4/stackoverflow/dataset/stackoverflow-test-100.csv
No changes to gs://qwiklabs-gcp-12b4b0f730830df4/stackoverflow/dataset/stackoverflow-test-20.csv
No changes to gs://qwiklabs-gcp-12b4b0f730830df4/stackoverf

## Packaging up the code

Take your code and put into a standard Python package structure.  <a href="taxifare/trainer/model.py">model.py</a> and <a href="taxifare/trainer/task.py">task.py</a> containing the Tensorflow code from earlier (explore the <a href="taxifare/trainer/">directory structure</a>).

In [7]:
%%bash
find ${MODEL_NAME}

stackoverflow
stackoverflow/trainer
stackoverflow/trainer/__init__.py
stackoverflow/trainer/task.py
stackoverflow/trainer/model.py
stackoverflow/trainer/model.pyc
stackoverflow/trainer/__init__.pyc


In [8]:
%%bash
cat ${MODEL_NAME}/trainer/model.py

#tf.estimator modeling

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import tensorflow as tf
import pandas as pd
import shutil
from google.datalab.ml import TensorBoard

print(tf.__version__)
tf.logging.set_verbosity(tf.logging.INFO)


CSV_COLUMNS = ['accepted', 'answer_count', 'comment_count', 'favorite_count', 'score', 'view_count', 'days_posted']
DEFAULTS = [[0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]

#DEFAULTS = [tf.constant([0], dtype=tf.int32),
#            tf.constant([0.0], dtype=tf.float32),
#            tf.constant([0.0], dtype=tf.float32),
#           tf.constant([0.0], dtype=tf.float32),
#            tf.constant([0.0], dtype=tf.float32),
#            tf.constant([0.0], dtype=tf.float32),
#            tf.constant([0.0], dtype=tf.float32) ]

#i=0
def read_dataset(filename, mode, batch_size = 512):
    def decode_line(row):
        #print(row)
        cols = tf.decode_csv(row, record_defaults = DEFAULTS)
   

## Running the Python module from the command-line

#### Clean model training dir/output dir

In [9]:
%%bash
# This is so that the trained model is started fresh each time. However, this needs to be done before 
# tensorboard is started
rm -rf $PWD/${TRAINING_DIR}

In [22]:
import google.datalab.bigquery as bq
import pandas as pd
import numpy as np
import seaborn as sns
import shutil

def sample_between(a, b, shredstart):
  basequery = """
  SELECT 
    answer_count, comment_count, favorite_count,  score, view_count,
    TIMESTAMP_DIFF(last_activity_date, creation_date, DAY) as days_posted,
    IF(accepted_answer_id IS NULL , 0, 1 ) as accepted
  FROM 
    `bigquery-public-data.stackoverflow.posts_questions`
  """
  
  # Use sampling for initial model development. Once model is developed, shread the entire dataset into  .csv files based on condition in the sampler.
  sampler = "WHERE MOD(ABS(FARM_FINGERPRINT(CAST(id as STRING))), EVERY_N * 100) < {1} AND MOD(ABS(FARM_FINGERPRINT(CAST(id as STRING))), EVERY_N * 100) >= {0}".format(
            shredstart, shredstart + 10
            )
  sampler2 = "AND {0} >= {1}\n AND {0} < {2}".format(
           "MOD(ABS(FARM_FINGERPRINT(CAST(id AS STRING))), EVERY_N * 100) * {}".format(10),
           (shredstart*10)+a, (shredstart*10)+b
          )
  return "{}\n{}\n{}".format(basequery, sampler, sampler2)


def create_query(phase, EVERY_N, shredstart):
  """Phase: train (70%) valid (15%) or test (15%)"""
  query = ""
  if phase == 'train':
    query = sample_between(0,60, shredstart)
  elif phase == 'valid':
    query = sample_between(60,75, shredstart)
  else:
    query = sample_between(75, 100, shredstart)
  return query.replace("EVERY_N", str(EVERY_N))

#print(create_query('train', 100))
#(answer_count - AVG(answer_count)) / STDDEV_POP(answer_count)  as answer_count,
#IF(accepted_answer_id IS NULL , cast(0 as int64), cast(1 as int64)) as accepted

def to_csv(df, filename):
  outdf = df.copy(deep = True)
  #outdf.loc[:, 'key'] = np.arange(0, len(outdf)) # rownumber as key
  # Reorder columns so that target is first column
  #print(outdf.head())
  #print(df.head())
  cols = outdf.columns.tolist()
  #print(cols)
  cols.remove('accepted')
  cols.insert(0, 'accepted')
  #print(cols)
  outdf = outdf[cols]  
  
  
  #Normalizing input columns  and replace NaN or null
  normalize_cols = outdf.columns.tolist()
  normalize_cols.remove('accepted')
  for normalize_cols_name in normalize_cols:
    outdf[normalize_cols_name].fillna(0, inplace = True)
    outdf[normalize_cols_name] = (outdf[normalize_cols_name] - outdf[normalize_cols_name].mean())  / outdf[normalize_cols_name].std() 
  #print(outdf)
  #print(outdf['answer_count'] )
  outdf.to_csv(filename,  header = False, index_label = False, index = False)
  print("Wrote {} to {}".format(len(outdf), filename))

for phase in ['train', 'valid', 'test']:
  #for x in range(2):
  for x in range(10):
    query = create_query(phase, 100, x*10)
    #print(query)
    df = bq.Query(query).execute().result().to_dataframe()
    #print(df.head())
    to_csv(df, 'stackoverflow-{}-{}.csv'.format(phase,(x+1)*10))


Wrote 10186 to stackoverflow-train-10.csv
Wrote 10333 to stackoverflow-train-20.csv
Wrote 10426 to stackoverflow-train-30.csv
Wrote 10260 to stackoverflow-train-40.csv
Wrote 10298 to stackoverflow-train-50.csv
Wrote 10401 to stackoverflow-train-60.csv
Wrote 10276 to stackoverflow-train-70.csv
Wrote 10249 to stackoverflow-train-80.csv
Wrote 10291 to stackoverflow-train-90.csv
Wrote 10332 to stackoverflow-train-100.csv
Wrote 3500 to stackoverflow-valid-10.csv
Wrote 3453 to stackoverflow-valid-20.csv
Wrote 3367 to stackoverflow-valid-30.csv
Wrote 3573 to stackoverflow-valid-40.csv
Wrote 3482 to stackoverflow-valid-50.csv
Wrote 3476 to stackoverflow-valid-60.csv
Wrote 3431 to stackoverflow-valid-70.csv
Wrote 3496 to stackoverflow-valid-80.csv
Wrote 3469 to stackoverflow-valid-90.csv
Wrote 3354 to stackoverflow-valid-100.csv
Wrote 3352 to stackoverflow-test-10.csv
Wrote 3476 to stackoverflow-test-20.csv
Wrote 3439 to stackoverflow-test-30.csv
Wrote 3408 to stackoverflow-test-40.csv
Wrote 35

In [11]:
%%bash
# Clear Cloud Storage bucket and copy the CSV files to Cloud Storage bucket
echo $BUCKET
gsutil -m rm -rf gs://${BUCKET}/${MODEL_NAME}/dataset/
gsutil -m cp ${PWD}/*.csv gs://${BUCKET}/${MODEL_NAME}/dataset/

qwiklabs-gcp-12b4b0f730830df4


CommandException: 1 files/objects could not be removed.
Copying file:///content/datalab/notebooks/stackoverflow-test-30.csv [Content-Type=text/csv]...
Copying file:///content/datalab/notebooks/stackoverflow-test-10.csv [Content-Type=text/csv]...
Copying file:///content/datalab/notebooks/stackoverflow-test-20.csv [Content-Type=text/csv]...
Copying file:///content/datalab/notebooks/stackoverflow-test-100.csv [Content-Type=text/csv]...
Copying file:///content/datalab/notebooks/stackoverflow-test-40.csv [Content-Type=text/csv]...
Copying file:///content/datalab/notebooks/stackoverflow-test-50.csv [Content-Type=text/csv]...
Copying file:///content/datalab/notebooks/stackoverflow-test-60.csv [Content-Type=text/csv]...
Copying file:///content/datalab/notebooks/stackoverflow-test-70.csv [Content-Type=text/csv]...
Copying file:///content/datalab/notebooks/stackoverflow-test-80.csv [Content-Type=text/csv]...
Copying file:///content/datalab/notebooks/stackoverflow-test-90.csv [Content-Type=text/c

In [12]:
# Ensure that we have TensorFlow 1.13.1 installed.
!pip3 freeze | grep tensorflow==1.13.1 || pip3 install tensorflow==1.13.1

    100% |████████████████████████████████| 92.5MB 220kB/s eta 0:00:01  0% |▎                               | 645kB 36.1MB/s eta 0:00:03    4% |█▎                              | 3.7MB 31.0MB/s eta 0:00:03    7% |██▍                             | 7.0MB 34.4MB/s eta 0:00:03    9% |███                             | 8.6MB 32.9MB/s eta 0:00:03    12% |████                            | 11.7MB 30.5MB/s eta 0:00:03    15% |█████                           | 14.7MB 31.0MB/s eta 0:00:03    17% |█████▋                          | 16.1MB 31.0MB/s eta 0:00:03    20% |██████▌                         | 18.9MB 33.0MB/s eta 0:00:03    36% |███████████▋                    | 33.5MB 29.3MB/s eta 0:00:03    39% |████████████▌                   | 36.2MB 26.9MB/s eta 0:00:03    40% |█████████████                   | 37.5MB 24.7MB/s eta 0:00:03    47% |███████████████                 | 43.5MB 10.8MB/s eta 0:00:05    53% |█████████████████               | 49.4MB 21.7MB/s eta 0:00:02    55% |██████████████████   

#### Monitor using Tensorboard

In [10]:
from google.datalab.ml import TensorBoard
TensorBoard().start('./'+ TRAINING_DIR)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorBoard was started successfully with pid 11137. Click here to access it.

11137

In [11]:
%%bash
# Setup python so it sees the task module which controls the model.py
export PYTHONPATH=${PYTHONPATH}:${PWD}/${MODEL_NAME}
# Currently set for python 2.  To run with python 3 
#    1.  Replace 'python' with 'python3' in the following command
#    2.  Edit trainer/task.py to reflect proper module import method 
python -m trainer.task \
   --train_data_paths=${PWD} \
   --eval_data_paths=${PWD}  \
   --output_dir=${PWD}/${TRAINING_DIR} \
   --train_steps=10 --job-dir=./tmp

1.13.1
[512]


/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Using config: {'_task_type': 'worker', '_evaluation_master': '', '_global_id_in_cluster': 0, '_eval_distribute': None, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_save_checkpoints_steps': 1000, '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f376475fdd8>, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_master': '', '_task_id': 0, '_num_worker_replicas': 1, '_keep_checkpoint_max': 5, '_device_fn': None, '_tf_random_seed': None, '_protocol': None, '_service': None, '_num_ps_r

In [12]:
%%bash
ls $PWD/${TRAINING_DIR}/export/exporter/

1558464829


In [13]:
%%writefile ./test.json
{"answer_count": 2, "comment_count": 2, "favorite_count": 1, "score": 2, "view_count": 2, "days_posted": 10}

Overwriting ./test.json


In [15]:
%%bash
# This model dir is the model exported after training and is used for prediction
#
model_dir=$(ls ${PWD}/${TRAINING_DIR}/export/exporter | tail -1)
# predict using the trained model
gcloud ml-engine local predict  \
    --model-dir=${PWD}/${TRAINING_DIR}/export/exporter/${model_dir} \
    --json-instances=./test.json

ERROR: (gcloud.ml-engine.local.predict) RuntimeError: Bad magic number in .pyc file



#### Stop Tensorboard
The training directory will be deleted.  Stop the exising tensorboard before removing the directory its using.

In [16]:
pids_df = TensorBoard.list()
if not pids_df.empty:
    for pid in pids_df['pid']:
        TensorBoard().stop(pid)
        print('Stopped TensorBoard with pid {}'.format(pid))

Stopped TensorBoard with pid 8130
Stopped TensorBoard with pid 11137


#### Clean model training dir/output dir

In [17]:
%%bash
# This is so that the trained model is started fresh each time. However, this needs to be done before 
# tensorboard is started
rm -rf $PWD/${TRAINING_DIR}

#### Restart tensorboard for monitoring

In [18]:
TensorBoard().start('./'+ TRAINING_DIR)

TensorBoard was started successfully with pid 11473. Click here to access it.

11473

## Running locally using gcloud

In [1]:
%%bash
# Use Cloud Machine Learning Engine to train the model in local file system
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${PWD}/${MODEL_NAME}/trainer \
   -- \
   --train_data_paths=${PWD} \
   --eval_data_paths=${PWD}  \
   --train_steps=10 \
   --output_dir=${PWD}/${TRAINING_DIR} 

/usr/local/envs/py3env/bin/python: Error while finding module specification for 'trainer.task' (ImportError: No module named 'trainer')


Use TensorBoard to examine results.  When I ran it (due to random seeds, your results will be different), the ```average_loss``` (Mean Squared Error) on the evaluation dataset was 187, meaning that the RMSE was around 13.

### Stop Tensorboard

In [33]:
pids_df = TensorBoard.list()
if not pids_df.empty:
    for pid in pids_df['pid']:
        TensorBoard().stop(pid)
        print('Stopped TensorBoard with pid {}'.format(pid))

Stopped TensorBoard with pid 7733


If the above step (to stop TensorBoard) appears stalled, just move on to the next step. You don't need to wait for it to return.

In [36]:
%%bash
ls $PWD/${TRAINING_DIR}

ls: cannot access '/content/datalab/notebooks/stackoverflow_trained': No such file or directory


## Submit training job using gcloud

First copy the training data to the cloud.  Then, launch a training job.

After you submit the job, go to the cloud console (http://console.cloud.google.com) and select <b>Machine Learning | Jobs</b> to monitor progress.  

<b>Note:</b> Don't be concerned if the notebook stalls (with a blue progress bar) or returns with an error about being unable to refresh auth tokens. This is a long-lived Cloud job and work is going on in the cloud.  Use the Cloud Console link (above) to monitor the job.

In [19]:
%%bash
# Clear Cloud Storage bucket and copy the CSV files to Cloud Storage bucket
echo $BUCKET
gsutil -m rm -rf gs://${BUCKET}/${MODEL_NAME}/dataset/
gsutil -m cp ${PWD}/*.csv gs://${BUCKET}/${MODEL_NAME}/dataset/

qwiklabs-gcp-12b4b0f730830df4


Removing gs://qwiklabs-gcp-12b4b0f730830df4/stackoverflow/dataset/stackoverflow-test-10.csv#1558463956219769...
Removing gs://qwiklabs-gcp-12b4b0f730830df4/stackoverflow/dataset/stackoverflow-test-100.csv#1558463956213080...
Removing gs://qwiklabs-gcp-12b4b0f730830df4/stackoverflow/dataset/stackoverflow-test-20.csv#1558463956169788...
Removing gs://qwiklabs-gcp-12b4b0f730830df4/stackoverflow/dataset/stackoverflow-test-30.csv#1558463956167974...
Removing gs://qwiklabs-gcp-12b4b0f730830df4/stackoverflow/dataset/stackoverflow-test-40.csv#1558463956198200...
Removing gs://qwiklabs-gcp-12b4b0f730830df4/stackoverflow/dataset/stackoverflow-test-50.csv#1558463956528845...
Removing gs://qwiklabs-gcp-12b4b0f730830df4/stackoverflow/dataset/stackoverflow-test-60.csv#1558463956707110...
Removing gs://qwiklabs-gcp-12b4b0f730830df4/stackoverflow/dataset/stackoverflow-test-70.csv#1558463956578898...
Removing gs://qwiklabs-gcp-12b4b0f730830df4/stackoverflow/dataset/stackoverflow-test-80.csv#15584639565

In [23]:
%%bash
OUTDIR=gs://${BUCKET}/${MODEL_NAME}/dataset/${TRAINING_DIR}
JOBNAME=${MODEL_NAME}_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
# Clear the Cloud Storage Bucket used for the training job
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/${MODEL_NAME}/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC \
   --runtime-version=$TFVERSION \
   -- \
   --train_data_paths="gs://${BUCKET}/${MODEL_NAME}/dataset" \
   --eval_data_paths="gs://${BUCKET}/${MODEL_NAME}/dataset"  \
   --output_dir=$OUTDIR \
   --train_steps=10000

gs://qwiklabs-gcp-12b4b0f730830df4/stackoverflow/dataset/stackoverflow_trained us-central1 stackoverflow_190521_191227
jobId: stackoverflow_190521_191227
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [stackoverflow_190521_191227] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe stackoverflow_190521_191227

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs stackoverflow_190521_191227


Don't be concerned if the notebook appears stalled (with a blue progress bar) or returns with an error about being unable to refresh auth tokens. This is a long-lived Cloud job and work is going on in the cloud. 

<b>Use the Cloud Console link to monitor the job and do NOT proceed until the job is done.</b>

## Deploy model

Find out the actual name of the subdirectory where the model is stored and use it to deploy the model.  Deploying model will take up to <b>5 minutes</b>.

In [24]:
%%bash
gsutil ls gs://${BUCKET}/${MODEL_NAME}/dataset/${TRAINING_DIR}/export/exporter

gs://qwiklabs-gcp-12b4b0f730830df4/stackoverflow/dataset/stackoverflow_trained/export/exporter/
gs://qwiklabs-gcp-12b4b0f730830df4/stackoverflow/dataset/stackoverflow_trained/export/exporter/1558466066/
gs://qwiklabs-gcp-12b4b0f730830df4/stackoverflow/dataset/stackoverflow_trained/export/exporter/1558466192/


#### Deploy model : step 1 - remove version info 
Before an existing cloud model can be removed, it must have any version info removed.  If an existing model does not exist, this command will generate an error but that is ok.

In [26]:
%%bash
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/${MODEL_NAME}/dataset/${TRAINING_DIR}/export/exporter | tail -1)

echo "MODEL_LOCATION = ${MODEL_LOCATION}"

gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}

MODEL_LOCATION = gs://qwiklabs-gcp-12b4b0f730830df4/stackoverflow/dataset/stackoverflow_trained/export/exporter/1558466192/


This will delete version [v1]...

Do you want to continue (Y/n)?  
ERROR: (gcloud.ml-engine.versions.delete) NOT_FOUND: Field: name Error: The model resource: "stackoverflow" was not found. Please create the Cloud ML model resource first by using 'gcloud ml-engine models create stackoverflow'.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: "The model resource: \"stackoverflow\" was not found. Please create\
      \ the Cloud ML model resource first by using 'gcloud ml-engine models create\
      \ stackoverflow'."
    field: name


#### Deploy model: step 2 - remove existing model
Now that the version info is removed from an existing model, the actual model can be removed.  If an existing model is not deployed, this command will generate an error but that is ok.  It just means the model with the given name is not deployed.

In [27]:
%%bash
gcloud ml-engine models delete ${MODEL_NAME}

This will delete model [stackoverflow]...

Do you want to continue (Y/n)?  
ERROR: (gcloud.ml-engine.models.delete) NOT_FOUND: Field: name Error: The model resource: "stackoverflow" was not found. Please create the Cloud ML model resource first by using 'gcloud ml-engine models create stackoverflow'.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: "The model resource: \"stackoverflow\" was not found. Please create\
      \ the Cloud ML model resource first by using 'gcloud ml-engine models create\
      \ stackoverflow'."
    field: name


#### Deploy model: step 3 - deploy new model

In [28]:
%%bash
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION

Created ml engine model [projects/qwiklabs-gcp-12b4b0f730830df4/models/stackoverflow].


#### Deploy model: step 4 - add version info to the new model

In [30]:
%%bash
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/${MODEL_NAME}/dataset/${TRAINING_DIR}/export/exporter | tail -1)

echo "MODEL_LOCATION = ${MODEL_LOCATION}"

gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

MODEL_LOCATION = gs://qwiklabs-gcp-12b4b0f730830df4/stackoverflow/dataset/stackoverflow_trained/export/exporter/1558466192/


Creating version (this might take a few minutes)......
......................................................................................................................................................................................................................................................................................................................done.


## Prediction

In [31]:
%%bash
gcloud ml-engine predict --model=${MODEL_NAME} --version=${MODEL_VERSION} --json-instances=./test.json

CLASS_IDS  CLASSES  LOGISTIC              LOGITS               PROBABILITIES
[1]        [u'1']   [0.8804934620857239]  [1.997111201286316]  [0.11950656771659851, 0.8804934620857239]


In [35]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1', credentials=credentials,
            discoveryServiceUrl='https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json')

request_data = {'instances':
  [
      {"answer_count": 0, 
       "comment_count": 2, 
       "favorite_count": 1, 
       "score": 2, 
       "view_count": 2, 
       "days_posted": 10
      }
  ]
}


parent = 'projects/%s/models/%s/versions/%s' % (PROJECT, MODEL_NAME, MODEL_VERSION)
response = api.projects().predict(body=request_data, name=parent).execute()
print("response={0}".format(response))

response={'predictions': [{'classes': ['1'], 'probabilities': [0.15197011828422546, 0.8480298519134521], 'logits': [1.7192319631576538], 'logistic': [0.8480298519134521], 'class_ids': [1]}]}


## Train on larger dataset

I have already followed the steps below and the files are already available. <b> You don't need to do the steps in this comment. </b> In the next chapter (on feature engineering), we will avoid all this manual processing by using Cloud Dataflow.

Go to http://bigquery.cloud.google.com/ and type the query:
<pre>
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
  'nokeyindata' AS key
FROM
  [nyc-tlc:yellow.trips]
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  AND ABS(HASH(pickup_datetime)) % 1000 == 1
</pre>

Note that this is now 1,000,000 rows (i.e. 100x the original dataset).  Export this to CSV using the following steps (Note that <b>I have already done this and made the resulting GCS data publicly available</b>, so you don't need to do it.):
<ol>
<li> Click on the "Save As Table" button and note down the name of the dataset and table.
<li> On the BigQuery console, find the newly exported table in the left-hand-side menu, and click on the name.
<li> Click on "Export Table"
<li> Supply your bucket name and give it the name train.csv (for example: gs://cloud-training-demos-ml/taxifare/ch3/train.csv). Note down what this is.  Wait for the job to finish (look at the "Job History" on the left-hand-side menu)
<li> In the query above, change the final "== 1" to "== 2" and export this to Cloud Storage as valid.csv (e.g.  gs://cloud-training-demos-ml/taxifare/ch3/valid.csv)
<li> Download the two files, remove the header line and upload it back to GCS.
</ol>

<p/>
<p/>

## Run Cloud training on 1-million row dataset

This took 60 minutes and uses as input 1-million rows.  The model is exactly the same as above. The only changes are to the input (to use the larger dataset) and to the Cloud MLE tier (to use STANDARD_1 instead of BASIC -- STANDARD_1 is approximately 10x more powerful than BASIC).  At the end of the training the loss was 32, but the RMSE (calculated on the validation dataset) was stubbornly at 9.03. So, simply adding more data doesn't help.

In [ ]:
%%bash

XXXXX  this takes 60 minutes. if you are sure you want to run it, then remove this line.

OUTDIR=gs://${BUCKET}/${MODEL_NAME}/${TRAINING_DIR}
JOBNAME=${MODEL_NAME}_$(date -u +%y%m%d_%H%M%S)
CRS_BUCKET=cloud-training-demos # use the already exported data
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/${MODEL_NAME}/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=STANDARD_1 \
   --runtime-version=$TFVERSION \
   -- \
   --train_data_paths="gs://${CRS_BUCKET}/${MODEL_NAME}/ch3/train.csv" \
   --eval_data_paths="gs://${CRS_BUCKET}/${MODEL_NAME}/ch3/valid.csv"  \
   --output_dir=$OUTDIR \
   --train_steps=100000   

## Challenge Exercise

Modify your solution to the challenge exercise in d_trainandevaluate.ipynb appropriately. Make sure that you implement training and deployment. Increase the size of your dataset by 10x since you are running on the cloud. Does your accuracy improve?

### Clean-up

#### Delete Model : step 1 - remove version info 
Before an existing cloud model can be removed, it must have any version info removed.  

In [ ]:
%%bash
gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}

#### Delete model: step 2 - remove existing model
Now that the version info is removed from an existing model, the actual model can be removed.  

In [ ]:
%%bash
gcloud ml-engine models delete ${MODEL_NAME}

Copyright 2016 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License